## Introduction of quantum portfolio optimization
*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

If you are an active investment manager who wants to invest $K$ dollars to $N$ projects, each with its return and risk, your goal is to find an optimal way to invest in the projects, taking into account the market impact and transaction costs.

To make the modeling easy to formulate, two assumptions are made to constrain the problem：
    1.Each asset is invested with an equal amount of money；
    2.Budget is a multiple of each investment amount and must be fully spent.

In the theory of portfolio optimization, the overall risk of a portfolio is related to the covariance between assets, which is proportional to the correlation coefficients of any two assets. The smaller the correlation coefficients, the smaller the covariance, and then the smaller the overall risk of the portfolio. Here we use the mean-variance approach to model this problem:
$$
\omega=\max _{x \in\{0,1\}^n} \mu^T x-q x^T S x \quad \text { subject to: } 1^T x=B,
$$
where each symbol has the following meaning:
- $x \in \{0, 1\}^{n}$ denotes the vector of binary decision variables, which indicate which each assets is picked ($x_i$=1) or not ($x_i=0$)
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets
- $S \in \mathbb{R}^{n \times n}$ represents the covariances between the assets
- $q > 0$ represents the risk factor of investment decision making
- $B$ denotes the budget, i.e. the number of assets to be selected out of $N$

Let us illustrate on the meaning of this equation. $\mu^T x$ describes the expected benefit of the investment plan represented by $x$. $x^T S x$ describes the correlation between the projects, which, after producting with the risk coefficient $q$, represents the risk incorporated in the investment plan. The restriction $1^T x=B$ requires the number of invested projects equals to our total budget. Therefore, $\omega$ represents the largest benefit we could get theoretically.

In order to find the optimal investment plan more easily, we can define the loss function
$$
C_x=q \sum_i \sum_j S_{j i} x_i x_j-\sum_i x_i \mu_i+A\left(B-\sum_i x_i\right)^2,
$$
where the restriction condition enters the function with the form of Lagrange multiplier. Therefore, our task becomes finding the investment plan $x$ that minimizes the loss $C_x$.

## Quantum encoding and solution

We now need to transform the cost function $C_x$ into a Hamiltonian to realize the encoding of the portfolio optimization problem. One just needs to do the following transformation:
$
x_i \mapsto \frac{I-Z_i}{2},
$
where $Z_i=I \otimes I \otimes \ldots \otimes Z \otimes \ldots \otimes I$, i.e., $Z_{i}$ is the Pauli operator acting solely on the $i$-th qubit. Thus using the above mapping, we can transform the cost function $C_x$ into a Hamiltonian $H_C$ for the system of $n$ qubits, the ground state of which represents the solution of the portfolio optimization problem. In order to find the ground state, we use the idea of variational quantum algorithms. We implement a parametric quantum circuit, and use it to generate a trial state $|\theta^* \rangle$. We use the quantum circuit to measure the expectation value of the Hamiltonian on this state. Then, classical gradient descent algorithm is implemented to adjust the parameters of the parametric circuit, where the expectation value evolves towards the ground state energy. After some iterations, we arrive at the optimal value
$$
|\theta^* \rangle  = \arg\min_\theta L(\vec{\theta})=\arg\min_\theta \left\langle\vec{\theta}\left|H_C\right| \vec{\theta}\right\rangle.
$$

Finally, we read out the probability distribution from the measurement result (i.e. decoding the quantum problem to give information about the original bit string)
$
p(z)=\left|\left\langle z \mid \vec{\theta}^*\right\rangle\right|^2.
$
In the case of quantum parameterized circuits with sufficient expressiveness, the greater the probability of a certain bit string, the greater the probability that it corresponds to an optimal solution to the portfolio optimization problem.

## User's guide
### Configuration file and input parameters
We provide a configuration file with previously chosen parameter. The user just needs to change the parameters in the `config.toml` file, and run `python qpo.py --config config.toml --logger qpo_log.log` in the terminal, to solve the portfolio optimization problem.
### Output
The results will be output to the `qpo_log.log` file. First of all, the process of optimization will be documented in the log. Users can see the evolution of loss function as the looping times increases. 
### Parameters
- `stock`, default is `'demo'`, i.e., using the stock data we provide in the demo file. Users can switch to `'random'` or `'custom'` to generate random stock data or use custom stock data. If user chooses to generate data randomly, the parameters `start_time` and `endtime` can be altered to specify the start and end date of the stock data. If user chooses to use custom data, he or she can store the information of the stock in a csv file, and write in the configuration file:

In [4]:
custom_data_path = 'file_name.csv'

### Online demonstration
Here, we provide an online demonstration version. Firstly, we define the parameters in the configuration file:

In [27]:
config_toml = r"""
# # The configuration file of quantum portfolio optimization problem.
# Use demo stock data
stock = 'demo' 
demo_data_path = 'demo_stock.csv'
# Number of investable projects
num_asset = 7
# Risk of decision making
risk_weight = 0.5
# Budget
budget = 0
# Penalty
penalty = 0
# The depth of the quantum circuit
circuit_depth = 2
# Number of loop cycles used in the optimization
iterations = 600
# Learning rate of gradient descent
learning_rate = 0.2
"""

The finance module in PaddleQuantum realizes the numerical simulation of the quantum portfolio optimization problem.

In [28]:
import os
import warnings
warnings.filterwarnings("ignore")
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import pandas as pd

import toml
from paddle_quantum.finance.qpo import portfolio_combination_optimization
from paddle_quantum.finance import DataSimulator

config = toml.loads(config_toml)
demo_data_path = config["demo_data_path"]
num_asset = config["num_asset"]
risk_weight = config["risk_weight"]
budget = config["budget"]
penalty = config["penalty"]
circuit_depth = config["circuit_depth"]
iterations = config["iterations"]
learning_rate = config["learning_rate"]

stocks_name = [("STOCK%s" % i) for i in range(num_asset)]
source_data = pd.read_csv(demo_data_path)
processed_data = [source_data['closePrice'+str(i)].tolist() for i in range(num_asset)]
data = DataSimulator(stocks_name)
data.set_data(processed_data)

invest = portfolio_combination_optimization(num_asset, data, iterations, learning_rate, risk_weight, budget,
                                       penalty, circuit=circuit_depth)
print(f"******************* The optimal investment plan is: {invest}  *******************")

100%|██████████| 600/600 [01:04<00:00,  9.24it/s]


******************* The optimal investment plan is: [2, 5, 6]  *******************


## Note
If the number of investable projects is small (`num_asset`$< 12$), we can diagonalize the Hamiltonian exactly, and compare the real minimum loss value with that found by the optimization process. If the difference is large, the optimization result may be unreliable, and re-choosing of the training parameters might be necessary. Finally, we output the optimal investment plan.

## References
```
@article{ORUS2019100028,
title = {Quantum computing for finance: Overview and prospects},
journal = {Reviews in Physics},
volume = {4},
pages = {100028},
year = {2019},
issn = {2405-4283},
doi = {https://doi.org/10.1016/j.revip.2019.100028},
url = {https://www.sciencedirect.com/science/article/pii/S2405428318300571},
author = {Román Orús and Samuel Mugel and Enrique Lizaso}
}

@ARTICLE{2020arXiv200614510E,
       author = {{Egger}, Daniel J. and {Gambella}, Claudio and {Marecek}, Jakub and {McFaddin}, Scott and {Mevissen}, Martin and {Raymond}, Rudy and {Simonetto}, Andrea and {Woerner}, Stefan and {Yndurain}, Elena},
        title = "{Quantum Computing for Finance: State of the Art and Future Prospects}",
      journal = {arXiv e-prints},
     keywords = {Quantum Physics, Quantitative Finance - Statistical Finance},
         year = 2020,
        month = jun,
          eid = {arXiv:2006.14510},
        pages = {arXiv:2006.14510},
archivePrefix = {arXiv},
       eprint = {2006.14510},
 primaryClass = {quant-ph},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2020arXiv200614510E},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}

@article{10.2307/2975974,
 ISSN = {00221082, 15406261},
 URL = {http://www.jstor.org/stable/2975974},
 author = {Harry Markowitz},
 journal = {The Journal of Finance},
 number = {1},
 pages = {77--91},
 publisher = {[American Finance Association, Wiley]},
 title = {Portfolio Selection},
 urldate = {2022-12-07},
 volume = {7},
 year = {1952}
}
```